# Imports

In [ ]:
import numpy as np
import requests
import json

#Comments about files


# Task Class

In [ ]:
class Task:
  '''
  The Task class represents an ARC task. 
  
  Attributes
  ----------
  train_pairs : list of Pair objects
    A list of training pairs for this ARC task.
  test_pairs : list of Pair objects
    A list of test pairs for this ARC task.

  Methods
  -------
  self.check_shapes()
    Prints the results of pair.check_shapes() for every pair in train_pairs and
    test_pairs. 
  Task.loadFromGithub(taskid)
    Creates a new Task object using the copy of ARC stored on Github to populate
    original_image_matrix for each Grid. The shapes attribute of each Grid 
    is initially set to None. 

  '''

  def __init__(self,train_pairs,test_pairs):
    self.train_pairs = train_pairs
    self.test_pairs = test_pairs

  def check_shapes(self):
    print('checking shapes for all pairs ...')
    for i in range(0,len(self.train_pairs)):
      print('train pair '+str(i))
      self.train_pairs[i].check_shapes()
    for i in range(0,len(self.test_pairs)):
      print('test pair '+str(i))
      self.test_pairs[i].check_shapes()

  def loadFromGithub(taskid):
    url = 'https://raw.githubusercontent.com/fchollet/ARC/master/data/training/'+taskid+'.json'
    response = requests.get(url)
    data = json.loads(response.text)
    pairs = {}
    for testOrTrain in ('test','train'):
      pairs[testOrTrain] = []
      for pair_number in range(len(data[testOrTrain])):
        input_grid = Grid(ImageMatrix(np.array(data[testOrTrain][pair_number]['input'])))
        output_grid = Grid(ImageMatrix(np.array(data[testOrTrain][pair_number]['output'])))
        pairs[testOrTrain].append(Pair(input_grid,output_grid))
    return(Task(pairs['train'],pairs['test']))


# Pair Class

In [ ]:
class Pair:
  '''
  The Pair class represents an ARC pair. 

  Attributes
  ----------
  input_grid : Grid object
    The input grid for this ARC pair.
  output_grid : Grid object
    The output grid for this ARC pair.

  Methods
  -------
  self.check_shapes()
    Prints the results of input_grid.check_shapes() and 
    output_grid.check_shapes().

  '''

  def __init__(self,input_grid,output_grid):
    self.input_grid = input_grid
    self.output_grid = output_grid

  def check_shapes(self):
    print('  input_grid',self.input_grid.check_shapes())
    print('  output_grid',self.output_grid.check_shapes())

# Grid Class

In [ ]:
class Grid:
  '''
  The Grid class represents an ARC grid. 
  
  Attributes
  ----------
  original_image_matrix : ImageMatrix object
    In the original corpus, a Grid is stored as a matrix. This attribute stores
    that original matrix as an ImageMatrix object.
  shapes : GroupOfShapes object
    A group of shapes that provides a precise definition of the grid using the
    Shape subclasses.
  
  Methods
  -------
  self.check_shapes()
    Returns None if shapes is None. Returns True if shapes.image_matrix is equal
    to original_image_matrix. Returns False otherwise.
  
  '''

  def __init__(self,original_image_matrix,shapes=None):
    self.original_image_matrix = original_image_matrix
    self.shapes = shapes

  def check_shapes(self):
    if self.shapes is None:
      return(None)
    return(self.shapes.image_matrix==self.original_image_matrix)

# ImageMatrix Class

In [ ]:
class ImageMatrix:
  '''
  The ImageMatrix class represents a matrix of colours, similar to the matrices 
  used to describe Grids in the original corpus. This class has three defining 
  attributes: matrix, x_offset, and y_offset. The matrix attribute stores the 
  actual matrix of colours. The x_offset and y_offset attributes give the 
  coordinates of the first element in the matrix. For example, a single dot at 
  x=5, y=3 with colour 9 would have the following attributes:
     matrix = np.array([[9]])
     x_offset = 5
     y_offset = 3
  
  Attributes
  ----------
  matrix : numpy array
    A two-dimensional numpy array. Each element in the array contains one of the
    following:
        - an integer from 0 to 9: corresponds to one of the colours in the
          original corpus
        - None: corresponds to transparency
        - the integer -1: when combined with another matrix using the combine 
          method, the -1 value will cut a hole in the matrix below it
  x_offset : int
    The x coordinate of the first element in the matrix.
  y_offset : int
    The y coordinate of the first element in the matrix.
  min_x : int
    The x coordinate of the first element in the matrix (equal to x_offset).
  min_y : int
    The y coordinate of the first element in the matrix (equal to y_offset).
  max_x : int
    The x coordinate of the last element in the matrix.
  max_y : int
    The y coordinate of the last element in the matrix.
  
  Methods
  -------
  self.getCell(x,y)
    Returns the value for cell x,y.
  self.updateCell(x,y,newValue)
    Changes the value for cell x,y to newValue.
  ImageMatrix.combine(list_of_image_matrices)
    Takes a list of ImageMatrix objects and combines them, such that the first
    object in the list is on the bottom and the last object is on top.

  '''

  def __init__(self,matrix,x_offset=0,y_offset=0):
    self.matrix = matrix
    self.x_offset = x_offset
    self.y_offset = y_offset
    self.min_x = x_offset
    self.max_x = len(matrix)+x_offset-1
    self.min_y = y_offset
    self.max_y = len(matrix[0])+y_offset-1

  def __eq__(self, other):
    if not(isinstance(other,ImageMatrix)):
      return(False)
    if self.x_offset!=other.x_offset or self.y_offset!=other.y_offset:
      return(False)
    return(np.array_equal(self.matrix,other.matrix))

  def getCell(self,x,y):
    if x<self.min_x or x>self.max_x or y<self.min_y or y>self.max_y:
      return(None)
    else:
      return(self.matrix[x-self.x_offset,y-self.y_offset])

  def updateCell(self,x,y,newValue):
    self.matrix[x-self.x_offset,y-self.y_offset] = newValue

  def combine(list_of_image_matrices):
    min_x = min([m.min_x for m in list_of_image_matrices])
    max_x = max([m.max_x for m in list_of_image_matrices])
    x_size = max_x-min_x+1
    min_y = min([m.min_y for m in list_of_image_matrices])
    max_y = max([m.max_y for m in list_of_image_matrices])
    y_size = max_y-min_y+1
    new_matrix = ImageMatrix(np.full((x_size,y_size),None))
    for m in list_of_image_matrices:
      for x in range(min_x,max_x+1):
        for y in range(min_y,max_y+1):
          m_value = m.getCell(x,y)
          if m_value is None:
            pass
          if m_value==-1:
            new_matrix.updateCell(x,y,None)
          if m_value in (0,1,2,3,4,5,6,7,8,9):
            new_matrix.updateCell(x,y,m_value)      
    return(new_matrix)

# Shape Class and Subclasses

In [ ]:
class Shape:
  pass

In [ ]:
class GroupOfShapes(Shape):

  def __init__(self,shapes):
    self.shapes = shapes
    self.image_matrix = ImageMatrix.combine([s.image_matrix for s in shapes])

In [ ]:
class Background(Shape):

  def __init__(self,x_size,y_size,colour): 
    self.x_size = x_size
    self.y_size = y_size
    self.colour = colour
    self.image_matrix = ImageMatrix(np.full((x_size,y_size),colour))

In [ ]:
class Dot(Shape):

    def __init__(self,x,y,colour):
      self.x = x
      self.y = y
      self.colour = colour
      self.image_matrix = ImageMatrix(np.full((1,1),colour),x,y)

In [ ]:
class RectangleFilled(Shape):

    def __init__(self,x1,y1,x2,y2,colour):
      self.x1 = x1
      self.y1 = y1
      self.x2 = x2
      self.y2 = y2
      self.colour = colour
      min_x = min(x1,x2)
      max_x = max(x1,x2)
      x_size = max_x-min_x+1
      min_y = min(y1,y2)
      max_y = max(y1,y2)
      y_size = max_y-min_y+1
      self.image_matrix = ImageMatrix(np.full((x_size,y_size),colour),min_x,min_y)

In [ ]:
class SquareFilled(Shape):

    def __init__(self,x1,y1,x2,y2,colour):
      self.x1 = x1
      self.y1 = y1
      self.x2 = x2
      self.y2 = y2
      self.colour = colour
      min_x = min(x1,x2)
      max_x = max(x1,x2)
      x_size = max_x-min_x+1
      min_y = min(y1,y2)
      max_y = max(y1,y2)
      y_size = max_y-min_y+1
      self.image_matrix = ImageMatrix(np.full((x_size,y_size),colour),min_x,min_y)

In [ ]:
class CheckeredLine(Shape):

    def __init__(self,x1,y1,x2,y2,colour1,colour2):
      self.x1 = x1
      self.y1 = y1
      self.x2 = x2
      self.y2 = y2
      self.colour1 = colour1
      self.colour2 = colour2

      min_x = min(x1,x2)
      max_x = max(x1,x2)
      x_size = max_x-min_x+1

      min_y = min(y1,y2)
      max_y = max(y1,y2)
      y_size = max_y-min_y+1

      #print(min_x,max_x,x_size,min_y,max_y,y_size)
      self.image_matrix = ImageMatrix(np.full((x_size,y_size),None),min_x,min_y)
      if x_size==1: # horizontal line
        count=0
        for y in range(min_y,max_y+1):
          if count%2 == 0:
            self.image_matrix.updateCell(min_x,y,colour1)
          else:
            self.image_matrix.updateCell(min_x,y,colour2)
          count+=1

      elif y_size==1: # vertical line
        count=0
        for x in range(min_x,max_x+1):
          if count%2 == 0:
            self.image_matrix.updateCell(x,min_y,colour1)
          else:
            self.image_matrix.updateCell(x,min_y,colour2)
          count+=1

In [ ]:
class CheckeredRectangle(Shape):
  
    def __init__(self,x1,y1,x2,y2,colour1,colour2):
      self.x1 = x1
      self.y1 = y1
      self.x2 = x2
      self.y2 = y2
      self.colour1 = colour1
      self.colour2 = colour2

      min_x = min(x1,x2)
      max_x = max(x1,x2)
      x_size = max_x-min_x+1

      min_y = min(y1,y2)
      max_y = max(y1,y2)
      y_size = max_y-min_y+1

      self.image_matrix = ImageMatrix(np.full((x_size,y_size),None),min_x,min_y)
      for x in range(min_x,max_x+1,2):
        count=0
        for y in range(min_y,max_y+1):
          if count%2 == 0:
            self.image_matrix.updateCell(x,y,colour1)
          else:
            self.image_matrix.updateCell(x,y,colour2)
          count+=1
      for x in range(min_x+1,max_x+1,2):
        count=0
        for y in range(min_y,max_y+1):
          if count%2 == 0:
            self.image_matrix.updateCell(x,y,colour2)
          else:
            self.image_matrix.updateCell(x,y,colour1)
          count+=1

In [ ]:
class RectangleOutline(Shape): 

    def __init__(self,x1,y1,x2,y2,colour):
      self.x1 = x1
      self.y1 = y1
      self.x2 = x2
      self.y2 = y2
      self.colour = colour
      min_x = min(x1,x2)
      max_x = max(x1,x2)
      x_size = max_x-min_x+1
      min_y = min(y1,y2)
      max_y = max(y1,y2)
      y_size = max_y-min_y+1
      self.image_matrix = ImageMatrix(np.full((x_size,y_size),None),min_x,min_y)
      for x in range(min_x,max_x+1):
        self.image_matrix.updateCell(x,min_y,colour)
        self.image_matrix.updateCell(x,max_y,colour)
      for y in range(min_y,max_y+1):
        self.image_matrix.updateCell(min_x,y,colour)
        self.image_matrix.updateCell(max_x,y,colour)

In [ ]:
class SquareOutline(Shape):

    def __init__(self,x1,y1,x2,y2,colour):
      self.x1 = x1
      self.y1 = y1
      self.x2 = x2
      self.y2 = y2
      self.colour = colour
      min_x = min(x1,x2)
      max_x = max(x1,x2)
      x_size = max_x-min_x+1
      min_y = min(y1,y2)
      max_y = max(y1,y2)
      y_size = max_y-min_y+1
      self.image_matrix = ImageMatrix(np.full((x_size,y_size),None),min_x,min_y)
      for x in range(min_x,max_x+1):
        self.image_matrix.updateCell(x,min_y,colour)
        self.image_matrix.updateCell(x,max_y,colour)
      for y in range(min_y,max_y+1):
        self.image_matrix.updateCell(min_x,y,colour)
        self.image_matrix.updateCell(max_x,y,colour)

In [ ]:
class Line(Shape):

    def __init__(self,x1,y1,x2,y2,colour):
      self.x1 = x1
      self.y1 = y1
      self.x2 = x2
      self.y2 = y2
      self.colour = colour
      min_x = min(x1,x2)
      max_x = max(x1,x2)
      x_size = max_x-min_x+1
      min_y = min(y1,y2)
      max_y = max(y1,y2)
      y_size = max_y-min_y+1
      
      #print(min_x,max_x,x_size,min_y,max_y,y_size)
      self.image_matrix = ImageMatrix(np.full((x_size,y_size),None),min_x,min_y)
      if x_size==1: # horizontal line
        for y in range(min_y,max_y+1):
          self.image_matrix.updateCell(min_x,y,colour)
      elif y_size==1: # vertical line
        for x in range(min_x,max_x+1):
          self.image_matrix.updateCell(x,min_y,colour)
      else: # diagonal line
        x = min_x
        y = min_y
        while (x<=max_x and y<=max_y):
          self.image_matrix.updateCell(x,y,colour)
          x += 1
          y += 1

# Example Task

In [ ]:
# Load task from Github

t1 = Task.loadFromGithub('445eab21')

# Define shapes for each grid

t1.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  RectangleOutline(0,1,3,4,7),
  RectangleOutline(5,3,8,7,8),
])

t1.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(2,2,8)
])

t1.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  RectangleOutline(0,0,3,4,6),
  RectangleOutline(5,2,8,7,7)
])

t1.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(2,2,7)
])

t1.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  RectangleOutline(0,1,6,6,4),
  RectangleOutline(7,7,9,9,2)
])

t1.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(2,2,4)
])

t1.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  RectangleOutline(0,0,8,4,3),
  RectangleOutline(0,6,9,9,9)
])

t1.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(2,2,3)
])

# Check shapes

t1.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
# Load task from Github

t2 = Task.loadFromGithub('67a423a3')

# Define shapes for each grid

t2.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(4,4,0),
  Dot(0,1,3),
  Line(1,0,1,3,2),
  Line(2,1,3,1,3)

])

t2.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(4,4,0),
  SquareOutline(0,0,2,2,4),
  Dot(1,1,2),
  Dot(1,3,2),
  Dot(3,1,3)
])

t2.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(8,8,0),
  Line(0,4,7,4,6),
  Line(4,0,4,3,8),
  Line(4,5,4,7,8)
])

t2.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(8,8,0),
  SquareOutline(3,3,5,5,4),
  Line(0,4,2,4,6),
  Line(4,0,4,2,8),
  Line(4,6,4,7,8),
  Dot(4,4,6),
  Line(6,4,7,4,6)
   
])

t2.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(6,6,0),
  Line(0,2,5,2,1),
  Line(2,0,2,1,9),
  Line(2,3,2,5,9)
])

t2.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(6,6,0),
  SquareOutline(1,1,3,3,4),
  Dot(0,2,1),
  Dot(2,0,9),
  Dot(2,2,1),
  Line(2,4,2,5,9),
  Line(4,2,5,2,1)
])

t2.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(12,12,0),
  Line(6,0,6,11,3),
  Line(0,8,11,8,5)
])

t2.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(12,12,0),
  SquareOutline(5,7,7,9,4),
  Dot(6,8,5),
  Line(0,8,4,8,5),
  Line(6,0,6,6,3),
  Line(6,10,6,11,3),
  Line(8,8,11,8,5)

])

# Check shapes

t2.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
# Load task from Github

t3 = Task.loadFromGithub('56ff96f3')

# Define shapes for each grid

t3.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  Dot(1,6,1),
  Dot(3,1,1),
  Dot(5,3,2),
  Dot(7,7,2)

])

t3.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  RectangleFilled(1,6,3,1,1),
  RectangleFilled(5,3,7,7,2)
])

t3.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(7,8,0),
  Dot(1,1,3),
  Dot(4,2,3),
  Dot(4,4,7),
  Dot(3,7,7)
])

t3.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(7,8,0),
  RectangleFilled(1,1,4,2,3),
  RectangleFilled(4,4,3,7,7)
])

t3.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  Dot(2,1,4),
  Dot(6,5,4)
])

t3.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  RectangleFilled(2,1,6,5,4)
  
])

t3.train_pairs[3].input_grid.shapes = GroupOfShapes([
  Background(6,11,0),
  Dot(1,1,7),
  Dot(3,7,7)
])

t3.train_pairs[3].output_grid.shapes = GroupOfShapes([
  Background(6,11,0),
  RectangleFilled(1,1,3,7,7)
])

t3.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(9,8,0),
  Dot(0,0,8),
  Dot(1,2,8),
  Dot(4,5,6),
  Dot(7,1,6)
])

t3.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(9,8,0),
  RectangleFilled(0,0,1,2,8),
  RectangleFilled(4,5,7,1,6)

])

# Check shapes

t3.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
train pair 3
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t4 = Task.loadFromGithub('a65b410d')

# Define shapes for each grid

t4.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(7,7,0),
  Line(3,0,3,1,2)

])

t4.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(7,7,0),
  Line(0,0,0,4,3),
  Line(1,0,1,3,3),
  Line(2,0,2,2,3),
  Line(3,0,3,1,2),
  Dot(4,0,1)
])

t4.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(8,9,0),
  Line(3,0,3,2,2)
])

t4.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(8,9,0),
  Line(0,0,0,5,3),
  Line(1,0,1,4,3),
  Line(2,0,2,3,3),
  Line(3,0,3,2,2),
  Line(4,0,4,1,1),
  Dot(5,0,1)
])

t4.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(7,9,0),
  Line(2,0,2,3,2)
])

t4.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(7,9,0),
  Line(0,0,0,5,3),
  Line(1,0,1,4,3),
  Line(2,0,2,3,2),
  Line(3,0,3,2,1),
  Line(4,0,4,1,1),
  Dot(5,0,1)
  
])

t4.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(9,9,0),
  Line(2,0,2,4,2)
])

t4.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(9,9,0),
  Line(0,0,0,6,3),
  Line(1,0,1,5,3),
  Line(2,0,2,4,2),
  Line(3,0,3,3,1),
  Line(4,0,4,2,1),
  Line(5,0,5,1,1),
  Dot(6,0,1)


])

# Check shapes

t4.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t5 = Task.loadFromGithub('dae9d2b5')

# Define shapes for each grid

t5.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(3,6,0),
  Dot(0,0,4),
  Dot(1,0,4),
  Dot(0,1,4),
  Dot(0,3,3),
  Dot(1,3,3),
  Dot(0,4,3),
  Dot(2,5,3)
])

t5.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,0,6),
  Dot(1,0,6),
  Dot(0,1,6),
  Dot(2,2,6)
])

t5.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(3,6,0),
  Dot(0,0,4),
  Dot(1,0,4),
  Dot(0,2,4),
  Dot(2,2,4),
  Dot(0,3,3),
  Dot(1,3,3),
  Dot(2,3,3),
  Dot(0,4,3)
])

t5.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,0,6),
  Dot(1,0,6),
  Dot(2,0,6),
  Dot(0,1,6),
  Dot(0,2,6),
  Dot(2,2,6)  
  
])

t5.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(3,6,0),
  Dot(2,0,4),
  Dot(1,1,4),
  Dot(2,1,4),
  Dot(0,2,4),
  Dot(1,2,4),
  Dot(1,3,3),
  Dot(0,4,3),
  Dot(1,5,3),
  Dot(2,5,3)
])

t5.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(1,0,6),
  Dot(2,0,6),
  Dot(0,1,6),
  Dot(1,1,6),
  Dot(2,1,6),
  Dot(0,2,6),
  Dot(1,2,6),
  Dot(2,2,6)
  
])

t5.train_pairs[3].input_grid.shapes = GroupOfShapes([
  Background(3,6,0),
  Dot(0,0,4),
  Dot(2,0,4),
  Dot(0,1,4),
  Dot(0,3,3),
  Dot(1,5,3)
])

t5.train_pairs[3].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,0,6),
  Dot(2,0,6),
  Dot(0,1,6),
  Dot(1,2,6)
  
])

t5.train_pairs[4].input_grid.shapes = GroupOfShapes([
  Background(3,6,0),
  Dot(1,0,4),
  Dot(2,2,4),
  Dot(2,3,3),
  Dot(0,4,3),
  Dot(2,4,3)
])

t5.train_pairs[4].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(1,0,6),
  Dot(2,0,6),
  Dot(0,1,6),
  Dot(2,1,6),
  Dot(2,2,6)
  
])

t5.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(3,6,0),
  Dot(1,0,4),
  Dot(0,1,4),
  Dot(2,1,4),
  Dot(0,2,4),
  Dot(0,3,3),
  Dot(1,3,3),
  Dot(2,3,3),
  Dot(1,4,3)
])

t5.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,0,6),
  Dot(1,0,6),
  Dot(2,0,6),
  Dot(0,1,6),
  Dot(1,1,6),
  Dot(2,1,6),
  Dot(0,2,6)

])

t5.test_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(3,6,0),
  Dot(2,0,4),
  Dot(1,1,4),
  Dot(0,2,4),
  Dot(1,3,3),
  Dot(2,3,3),
  Dot(0,4,3),
  Dot(1,4,3),
  Dot(1,5,3)
])

t5.test_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(1,0,6),
  Dot(2,0,6),
  Dot(1,1,6),
  Dot(0,1,6),
  Dot(0,2,6),
  Dot(1,2,6)

])

# Check shapes

t5.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
train pair 3
  input_grid True
  output_grid True
train pair 4
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True
test pair 1
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t6 = Task.loadFromGithub('e9afcf9a')

# Define shapes for each grid

t6.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(2,6,0),
  Line(0,0,0,5,3),
  Line(1,0,1,5,9)

])

t6.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(2,6,0),
  CheckeredRectangle(0,0,1,5,3,9)
])

t6.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(2,6,0),
  Line(0,0,0,5,4),
  Line(1,0,1,5,8)
])

t6.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(2,6,0),
  CheckeredRectangle(0,0,1,5,4,8)
])

t6.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(2,6,0),
  Line(0,0,0,5,6),
  Line(1,0,1,5,2)
])

t6.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(2,6,0),
  CheckeredRectangle(0,0,1,5,6,2)
])

# Check shapes

t6.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t7 = Task.loadFromGithub('ba26e723')

# Define shapes for each grid

t7.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(3,10,0),
  CheckeredLine(0,0,0,9,4,0),
  Line(1,0,1,9,4),
  CheckeredLine(2,0,2,9,0,4)

])

t7.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(3,10,0),
  CheckeredLine(0,0,0,9,4,0),
  Line(1,0,1,9,4),
  CheckeredLine(2,0,2,9,0,4),
  Line(0,0,1,0,6),
  Line(1,3,2,3,6),
  Line(0,6,1,6,6),
  Line(1,9,2,9,6)
])

t7.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(3,11,0),
  CheckeredLine(0,0,0,10,0,4),
  Line(1,0,1,10,4),
  CheckeredLine(2,0,2,10,4,0)
])

t7.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(3,11,0),
  CheckeredLine(0,0,0,10,0,4),
  Line(1,0,1,10,4),
  CheckeredLine(2,0,2,10,4,0),
  Line(1,0,2,0,6),
  Line(0,3,1,3,6),
  Line(1,6,2,6,6),
  Line(0,9,1,9,6)
])

t7.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(3,11,0),
  CheckeredLine(0,0,0,10,4,0),
  Line(1,0,1,10,4),
  CheckeredLine(2,0,2,10,0,4)
])

t7.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(3,11,0),
  CheckeredLine(0,0,0,10,4,0),
  Line(1,0,1,10,4),
  CheckeredLine(2,0,2,10,0,4),
  Line(0,0,1,0,6),
  Line(1,3,2,3,6),
  Line(0,6,1,6,6),
  Line(1,9,2,9,6)
])

t7.train_pairs[3].input_grid.shapes = GroupOfShapes([
  Background(3,13,0),
  CheckeredLine(0,0,0,12,4,0),
  Line(1,0,1,12,4),
  CheckeredLine(2,0,2,12,0,4)
])

t7.train_pairs[3].output_grid.shapes = GroupOfShapes([
  Background(3,13,0),
  CheckeredLine(0,0,0,12,4,0),
  Line(1,0,1,12,4),
  CheckeredLine(2,0,2,12,0,4),
  Line(0,0,1,0,6),
  Line(1,3,2,3,6),
  Line(0,6,1,6,6),
  Line(1,9,2,9,6),
  Line(0,12,1,12,6)
])

t7.train_pairs[4].input_grid.shapes = GroupOfShapes([
  Background(3,14,0),
  CheckeredLine(0,0,0,13,0,4),
  Line(1,0,1,13,4),
  CheckeredLine(2,0,2,13,4,0)
])

t7.train_pairs[4].output_grid.shapes = GroupOfShapes([
  Background(3,14,0),
  CheckeredLine(0,0,0,13,0,4),
  Line(1,0,1,13,4),
  CheckeredLine(2,0,2,13,4,0),
  Line(1,0,2,0,6),
  Line(0,3,1,3,6),
  Line(1,6,2,6,6),
  Line(0,9,1,9,6),
  Line(1,12,2,12,6)
])

t7.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(3,17,0),
  CheckeredLine(0,0,0,16,0,4),
  Line(1,0,1,16,4),
  CheckeredLine(2,0,2,16,4,0)
])

t7.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(3,17,0),
  CheckeredLine(0,0,0,16,0,4),
  Line(1,0,1,16,4),
  CheckeredLine(2,0,2,16,4,0),
  Line(1,0,2,0,6),
  Line(0,3,1,3,6),
  Line(1,6,2,6,6),
  Line(0,9,1,9,6),
  Line(1,12,2,12,6),
  Line(0,15,1,15,6)

])

# Check shapes

t7.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
train pair 3
  input_grid True
  output_grid True
train pair 4
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t8 = Task.loadFromGithub('9f236235')

# Define shapes for each grid

t8.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(19,19,0),
  Line(0,4,18,4,2),
  Line(0,9,18,9,2),
  Line(0,14,18,14,2),
  Line(4,0,4,18,2),
  Line(9,0,9,18,2),
  Line(14,0,14,18,2),
  SquareFilled(0,0,3,3,3),
  SquareFilled(5,5,8,8,3),
  SquareFilled(10,10,13,13,3),
  SquareFilled(15,10,18,13,3),
  SquareFilled(15,5,18,8,3),
  SquareFilled(15,0,18,3,3)

])

t8.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(4,4,0),
  Dot(0,3,3),
  Dot(1,2,3),
  Dot(2,1,3),
  Dot(3,1,3),
  Dot(3,2,3),
  Dot(3,3,3)
  
])

t8.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(19,19,0),
  Line(0,4,18,4,8),
  Line(0,9,18,9,8),
  Line(0,14,18,14,8),
  Line(4,0,4,18,8),
  Line(9,0,9,18,8),
  Line(14,0,14,18,8),
  SquareFilled(5,0,8,3,2),
  SquareFilled(0,5,3,8,2),
  SquareFilled(5,5,8,8,1),
  SquareFilled(10,10,13,13,1),
  SquareFilled(15,15,18,18,3)
])

t8.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(4,4,0),
  Dot(0,2,2),
  Dot(1,2,1),
  Dot(1,3,2),
  Dot(2,1,1),
  Dot(3,0,3)
])

t8.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(11,11,0),
  Line(0,3,10,3,2),
  Line(0,7,10,7,2),
  Line(3,0,3,10,2),
  Line(7,0,7,10,2),
  SquareFilled(0,4,2,6,8),
  SquareFilled(4,4,6,6,8),
  SquareFilled(4,0,6,2,8),
  SquareFilled(8,8,10,10,4)
])

t8.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,1,8),
  Dot(1,1,8),
  Dot(1,2,8),
  Dot(2,0,4)
])

t8.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(23,23,0),
  Line(0,5,22,5,8),
  Line(0,11,22,11,8),
  Line(0,17,22,17,8),
  Line(5,0,5,22,8),
  Line(11,0,11,22,8),
  Line(17,0,17,22,8),
  SquareFilled(0,0,4,4,1),
  SquareFilled(0,6,4,10,3),
  SquareFilled(0,12,4,16,1),
  SquareFilled(0,18,4,22,1),
  SquareFilled(6,6,10,10,3),
  SquareFilled(12,0,16,4,2),
  SquareFilled(12,6,16,10,3),
  SquareFilled(12,18,16,22,2),
  SquareFilled(18,6,22,10,3)

])

t8.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(4,4,0),
  Dot(0,0,1),
  Dot(0,1,1),
  Dot(0,3,1),
  Dot(0,2,3),
  Dot(1,2,3),
  Dot(2,2,3),
  Dot(3,2,3),
  Dot(2,0,2),
  Dot(2,3,2)

])

# Check shapes

t8.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t9 = Task.loadFromGithub('6f8cd79b')

# Define shapes for each grid

t9.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(3,3,0)

])

t9.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  RectangleOutline(0,0,2,2,8)
  
])

t9.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(4,3,0)
])

t9.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(4,3,0),
  RectangleOutline(0,0,3,2,8)
])

t9.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(5,4,0)

])

t9.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(5,4,0),
  RectangleOutline(0,0,4,3,8)
])

t9.train_pairs[3].input_grid.shapes = GroupOfShapes([
  Background(5,6,0)
])

t9.train_pairs[3].output_grid.shapes = GroupOfShapes([
  Background(5,6,0),
  RectangleOutline(0,0,4,5,8)
])

t9.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(7,6,0)

])

t9.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(7,6,0),
  RectangleOutline(0,0,6,5,8)

])

# Check shapes

t9.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
train pair 3
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t10 = Task.loadFromGithub('d037b0a7')

# Define shapes for each grid

t10.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(2,0,3),
  Dot(1,1,4),
  Dot(0,2,6)
  
])

t10.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Line(2,0,2,0,3),
  Line(1,1,2,1,4),
  Line(0,2,2,2,6)
  
])

t10.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(1,0,7),
  Dot(0,1,2),
  Dot(1,2,8)

])

t10.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Line(1,0,2,0,7),
  Line(0,1,2,1,2),
  Line(1,2,2,2,8)
  
])

t10.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,0,4),
  Dot(1,1,2)
])

t10.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Line(0,0,2,0,4),
  Line(1,1,2,1,2)
  
])

t10.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,0,4),
  Dot(2,1,7),
  Dot(0,2,8)


])

t10.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Line(2,1,2,1,7),
  Line(0,0,2,0,4),
  Line(0,2,2,2,8)

])

# Check shapes

t10.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t11 = Task.loadFromGithub('a85d4709')

# Define shapes for each grid

t11.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(2,0,5),
  Dot(1,1,5),
  Dot(0,2,5)

])

t11.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Line(0,0,0,2,3),
  Line(1,0,1,2,4),
  Line(2,0,2,2,2)
  
])

t11.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,2,5),
  Dot(1,2,5),
  Dot(2,2,5)
])

t11.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Line(0,0,0,2,3),
  Line(1,0,1,2,3),
  Line(2,0,2,2,3)
  
])

t11.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,0,5),
  Dot(1,1,5),
  Dot(2,0,5)
])

t11.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Line(0,0,0,2,2),
  Line(1,0,1,2,4),
  Line(2,0,2,2,2)
  
])

t11.train_pairs[3].input_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,1,5),
  Dot(1,2,5),
  Dot(2,1,5)
])

t11.train_pairs[3].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Line(0,0,0,2,4),
  Line(1,0,1,2,3),
  Line(2,0,2,2,4)
  
])

t11.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Dot(0,2,5),
  Dot(1,0,5),
  Dot(2,1,5)

])

t11.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(3,3,0),
  Line(0,0,0,2,3),
  Line(1,0,1,2,2),
  Line(2,0,2,2,4)

  

])

# Check shapes

t11.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
train pair 3
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t12 = Task.loadFromGithub('dc1df850')

# Define shapes for each grid

t12.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(5,5,0),
  Dot(0,0,2),
  Dot(1,3,2),
  Dot(3,1,6)

])

t12.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(5,5,0),
  Dot(0,0,2),
  Dot(1,3,2),
  Dot(3,1,6),
  Dot(0,1,1),
  Dot(1,0,1),
  Dot(1,1,1),
  SquareOutline(0,2,2,4,1),

  
])

t12.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(8,8,0),
  Dot(0,7,2),
  Dot(6,2,2),
  Dot(2,3,3),
  Dot(4,6,8)
])

t12.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(8,8,0),
  Dot(0,7,2),
  Dot(6,2,2),
  Dot(2,3,3),
  Dot(4,6,8),
  Dot(0,6,1),
  Dot(1,6,1),
  Dot(1,7,1),
  SquareOutline(5,1,7,3,1)
  
])

t12.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(4,5,0),
  Dot(1,1,2)
])

t12.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(4,5,0),
  Dot(1,1,2),
  SquareOutline(0,0,2,2,1)
  
])

t12.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  Dot(1,2,2),
  Dot(3,7,2),
  Dot(7,5,2),
  Dot(0,8,7),
  Dot(5,1,7),
  Dot(9,9,5)

])

t12.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  Dot(1,2,2),
  Dot(3,7,2),
  Dot(7,5,2),
  Dot(0,8,7),
  Dot(5,1,7),
  Dot(9,9,5),
  SquareOutline(0,1,2,3,1),
  SquareOutline(2,6,4,8,1),
  SquareOutline(6,4,8,6,1)

  

])

# Check shapes

t12.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid True
  output_grid True
train pair 1
  input_grid True
  output_grid True
train pair 2
  input_grid True
  output_grid True
test pair 0
  input_grid True
  output_grid True


In [ ]:
from IPython.lib.display import DisplayObject
# Load task from Github

t13 = Task.loadFromGithub('1caeab9d')

# Define shapes for each grid

t13.train_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(5,5,0),
  Dot(0,0,2),
  Dot(1,3,2),
  Dot(3,1,6)

]) 

t13.train_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(5,5,0),
  Dot(0,0,2),
  Dot(1,3,2),
  Dot(3,1,6),
  Dot(0,1,1),
  Dot(1,0,1),
  Dot(1,1,1),
  SquareOutline(0,2,2,4,1),

  
])

t13.train_pairs[1].input_grid.shapes = GroupOfShapes([
  Background(8,8,0),
  Dot(0,7,2),
  Dot(6,2,2),
  Dot(2,3,3),
  Dot(4,6,8)
])

t13.train_pairs[1].output_grid.shapes = GroupOfShapes([
  Background(8,8,0),
  Dot(0,7,2),
  Dot(6,2,2),
  Dot(2,3,3),
  Dot(4,6,8),
  Dot(0,6,1),
  Dot(1,6,1),
  Dot(1,7,1),
  SquareOutline(5,1,7,3,1)
  
])

t13.train_pairs[2].input_grid.shapes = GroupOfShapes([
  Background(4,5,0),
  Dot(1,1,2)
])

t13.train_pairs[2].output_grid.shapes = GroupOfShapes([
  Background(4,5,0),
  Dot(1,1,2),
  SquareOutline(0,0,2,2,1)
  
])

t13.test_pairs[0].input_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  Dot(1,2,2),
  Dot(3,7,2),
  Dot(7,5,2),
  Dot(0,8,7),
  Dot(5,1,7),
  Dot(9,9,5)

])

t13.test_pairs[0].output_grid.shapes = GroupOfShapes([
  Background(10,10,0),
  Dot(1,2,2),
  Dot(3,7,2),
  Dot(7,5,2),
  Dot(0,8,7),
  Dot(5,1,7),
  Dot(9,9,5),
  SquareOutline(0,1,2,3,1),
  SquareOutline(2,6,4,8,1),
  SquareOutline(6,4,8,6,1)

  

])

# Check shapes

t13.check_shapes()

checking shapes for all pairs ...
train pair 0
  input_grid False
  output_grid False
train pair 1
  input_grid False
  output_grid False
train pair 2
  input_grid False
  output_grid False
test pair 0
  input_grid False
  output_grid False
